In [9]:
# import pandas as pd
# X_meta = pd.read_csv("../data/training/X_meta.csv")
# X_meta.head()

In [2]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1/255
)

train_generator = train_datagen.flow_from_directory(
    '../data/images_cropped/',
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    '../data/images_cropped/',
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical'
)

Using TensorFlow backend.


Found 2975 images belonging to 3 classes.
Found 2975 images belonging to 3 classes.


In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.losses import categorical_crossentropy


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss=categorical_crossentropy,
              optimizer='rmsprop',
              metrics=['accuracy'])

In [4]:
import pathlib

sample_size = len(list(pathlib.Path('../data/images_cropped/').rglob('./*')))
batch_size = 16

In [8]:
model.fit_generator(
    train_generator,
    steps_per_epoch=sample_size // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=round(sample_size * 0.2)
)

Epoch 1/50
 10/186 [>.............................] - ETA: 2:47 - loss: 1.0169 - acc: 0.4000

KeyboardInterrupt: 

In [6]:
model.to_csv("/opt/ml/model/clf.h5")